In [1665]:
import numpy as np
import pandas as pd
import pandas_bokeh;

In [1666]:
# Embedding plots in Jupyter/Colab Notebook
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [1667]:
# For exporting plots as HTML
pandas_bokeh.output_file(filename='birds_dashboard.html',title='Birds Dashboard')

In [1668]:
# Get data from google sheets
sheet_id = '1dENiPTJtdqOiU9Oiwxen2UU2K4HP1jOR_MyGcjQP81k'
sheet_name = 'registro'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df = pd.read_csv(url, usecols=[0,2,3,4])

In [1669]:
# Get datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         187 non-null    object 
 1   Description  187 non-null    object 
 2   Qty (g)      187 non-null    float64
 3   Cost($)      399 non-null    float64
dtypes: float64(2), object(2)
memory usage: 12.6+ KB


In [1670]:
# Change datatypes
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month

In [1671]:
# Data for >>Pie Plot<< and >>Scatter Plot<<
df.head()

,Date,Description,Qty (g),Cost($),Month
0,2021-03-01,Papaya,227.0,385.9,3.0
1,2021-03-01,Plantain,165.0,379.5,3.0
2,2021-03-02,Plantain,164.0,377.2,3.0
3,2021-03-03,Banana,148.0,355.2,3.0
4,2021-03-03,Banana,124.0,297.6,3.0


In [1672]:
# Group by month -> Data for >>Simple Line Plot<< and >>Bar Plot<<

by_month = df.groupby(['Month']).sum()
by_month = by_month.rename(index={
    1.0:'Jan', 2.0:'Feb', 3.0:'Mar', 4.0:'Apr', 5.0:'May', 6.0:'Jun',
    7.0:'Jun', 8.0:'Ago', 9.0:'Sep', 10.0:'Oct', 11.0:'Nov', 12.0:'Dic'})
by_month.head()

,Qty (g),Cost($)
Month,,
Mar,4500.0,10207.9
Apr,3195.0,7317.9
May,6639.0,16558.1
Jun,5680.0,13177.6
Jun,7780.0,17926.7


In [1673]:
# Group by description and month
by_fruit = df.groupby(['Month','Description']).sum()
by_fruit.head()

Qty (g)  Cost($)
Month Description                  
3.0   Banana        1273.0   3055.2
      Papaya         449.0    763.3
      Plantain      2778.0   6389.4
4.0   Banana         396.0    950.4
      Papaya         117.0    198.9

In [1674]:
# Subset by qty -> Data for >>Stacked Bar Plot<<
by_fruit = by_fruit.reset_index()
df_qty = by_fruit.drop(['Cost($)'], axis=1)
df_qty.head()


,Month,Description,Qty (g)
0,3.0,Banana,1273.0
1,3.0,Papaya,449.0
2,3.0,Plantain,2778.0
3,4.0,Banana,396.0
4,4.0,Papaya,117.0


In [1675]:
# Pivot subset qty
df_qty = df_qty.pivot(columns='Description', index='Month', values='Qty (g)')
df_qty = df_qty.rename(index={
    1.0:'Jan', 2.0:'Feb', 3.0:'Mar', 4.0:'Apr', 5.0:'May', 6.0:'Jun',
    7.0:'Jun', 8.0:'Ago', 9.0:'Sep', 10.0:'Oct', 11.0:'Nov', 12.0:'Dic'})
df_qty= df_qty.groupby('Month', sort=False).sum()
df_qty.head()

Description,Banana,Orange,Papaya,Plantain
Month,,,,
Mar,1273.0,0.0,449.0,2778.0
Apr,396.0,0.0,117.0,2682.0
May,799.0,1097.0,1094.0,3649.0
Jun,1463.0,0.0,0.0,11997.0
Ago,0.0,0.0,0.0,5563.0


In [1676]:
#Subset by cost -> Data fo >>Multiline Plot<<
df_cost = by_fruit.drop(['Qty (g)'], axis=1)
df_cost.head()

,Month,Description,Cost($)
0,3.0,Banana,3055.2
1,3.0,Papaya,763.3
2,3.0,Plantain,6389.4
3,4.0,Banana,950.4
4,4.0,Papaya,198.9


In [1677]:
# Pivot subset cost
df_cost = df_cost.pivot(columns='Description', index='Month', values='Cost($)')
df_cost = df_cost.rename(index={
    1.0:'Jan', 2.0:'Feb', 3.0:'Mar', 4.0:'Apr', 5.0:'May', 6.0:'Jun',
    7.0:'Jun', 8.0:'Ago', 9.0:'Sep', 10.0:'Oct', 11.0:'Nov', 12.0:'Dic'})
df_cost = df_cost.groupby('Month', sort=False).sum()
df_cost.head()

Description,Banana,Orange,Papaya,Plantain
Month,,,,
Mar,3055.2,0.0,763.3,6389.4
Apr,950.4,0.0,198.9,6168.6
May,1917.6,4388.0,1859.8,8392.7
Jun,3511.2,0.0,0.0,27593.1
Ago,0.0,0.0,0.0,12794.9


# Plotting

In [1678]:
# Plot 1. Simple Line Plot
line_plot= by_month.plot_bokeh(kind='line', y='Qty (g)', color='#6C3FE3', plot_data_points=True, legend=False , xlabel='' ,show_figure=False)

In [1679]:
# Plot 2. Pie Plot
pie_plot= df.groupby(['Description']).sum().plot_bokeh.pie(y='Qty (g)',colormap='Colorblind', title= 'Fruit eaten by birds in my window in 2021',show_figure=False)

__x__values_original


In [1680]:
# Plot 3. Stacked Bar Plot
stackedbar_plot = df_qty.plot_bokeh(kind='bar', stacked=True, colormap='Colorblind' ,xlabel='', show_figure=False)

In [1681]:
# Plot 4. Multiline Plot
multiline_plot = df_cost.plot_bokeh(kind='line', colormap='Colorblind', ylabel='Cost($)', show_figure=False)

In [1682]:
# Plot 5. Bar Plot
bar_plot = by_month.plot_bokeh(kind="bar", y='Cost($)', ylabel='', color='#8561E5', legend=False, show_figure=False)

In [1683]:
# Plot 6. Scatter Plot
scatter_plot = df.plot_bokeh(kind="scatter", x='Qty (g)', y='Cost($)', ylabel='',category='Description', legend = "bottom_right", colormap='Bokeh',show_figure=False)

In [1684]:
#Make Dashboard with Grid Layout: 
pandas_bokeh.plot_grid([[line_plot, pie_plot, stackedbar_plot], 
    [multiline_plot, bar_plot, scatter_plot]], 
    width=600, toolbar_location="right")

Row(id='80517', ...)